In [2]:
!pip install torch torchvision torchaudio
!pip install torch-geometric
!pip install flask


     ---------------------------------------- 0.0/63.1 kB ? eta -:--:--
     ------ --------------------------------- 10.2/63.1 kB ? eta -:--:--
     ------------ ------------------------- 20.5/63.1 kB 217.9 kB/s eta 0:00:01
     ------------ ------------------------- 20.5/63.1 kB 217.9 kB/s eta 0:00:01
     ------------------------ ------------- 41.0/63.1 kB 196.9 kB/s eta 0:00:01
     -------------------------------------- 63.1/63.1 kB 260.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.1 MB 1.3 MB/s eta 0:00:01
   -- ------------------------------------- 0.1/1.1 MB 812.7 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.1 MB 812.7 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.1 MB 595.3 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.1 MB 762.6 kB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.1 MB 737.3 kB/s eta 0:00:02


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
urduhack 1.1.1 requires Click~=7.1, but you have click 8.1.7 which is incompatible.


In [2]:
# Import necessary libraries
import torch
from torch_geometric.data import Data
import networkx as nx
import random
import numpy as np
from sklearn.model_selection import train_test_split

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

def generate_synthetic_graph(num_nodes=100, edge_prob=0.05, seed=42):
    G = nx.erdos_renyi_graph(n=num_nodes, p=edge_prob, seed=seed)
    while not nx.is_connected(G):
        G = nx.erdos_renyi_graph(n=num_nodes, p=edge_prob, seed=random.randint(0, 10000))
    return G

def preprocess_data(G, test_size=0.2, seed=42):
    # Extract edges
    edges = list(G.edges())
    num_edges = len(edges)
    
    # Split edges into train and test
    train_edges, test_edges = train_test_split(edges, test_size=test_size, random_state=seed)
    
    # Create negative samples for testing
    def is_not_edge(u, v, G):
        return not G.has_edge(u, v) and u != v
    
    test_neg_edges = []
    while len(test_neg_edges) < len(test_edges):
        u = random.randint(0, G.number_of_nodes() - 1)
        v = random.randint(0, G.number_of_nodes() - 1)
        if is_not_edge(u, v, G):
            test_neg_edges.append((u, v))
    
    # Convert to tensors
    edge_index = torch.tensor(train_edges + test_edges, dtype=torch.long).t().contiguous()
    
    # Node features: identity matrix (one-hot encoding)
    num_nodes = G.number_of_nodes()
    x = torch.eye(num_nodes, dtype=torch.float)
    
    data = Data(x=x, edge_index=edge_index)
    data.train_edges = torch.tensor(train_edges, dtype=torch.long)
    data.test_edges = torch.tensor(test_edges, dtype=torch.long)
    data.test_neg_edges = torch.tensor(test_neg_edges, dtype=torch.long)
    
    return data

# Generate and preprocess the graph
G = generate_synthetic_graph()
data = preprocess_data(G)

print("Data preprocessing completed.")
print(f"Number of nodes: {data.num_nodes}")
print(f"Number of training edges: {data.train_edges.size(0)}")
print(f"Number of testing edges: {data.test_edges.size(0)}")


Data preprocessing completed.
Number of nodes: 100
Number of training edges: 179
Number of testing edges: 45


In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GNN15Layer(nn.Module):
    def __init__(self, num_features, hidden_dim):
        super(GNN15Layer, self).__init__()
        self.convs = nn.ModuleList()
        self.convs.append(GCNConv(num_features, hidden_dim))
        for _ in range(14):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
        self.dropout = 0.5
        self.linear = nn.Linear(hidden_dim * 2, 1)  # For link prediction

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        return x

    def decode(self, z, edge_pairs):
        # z: node embeddings
        # edge_pairs: [2, num_edges]
        src = z[edge_pairs[0]]
        dst = z[edge_pairs[1]]
        return torch.sigmoid((src * dst).sum(dim=1))


In [4]:
from torch.optim import Adam
from sklearn.metrics import roc_auc_score, average_precision_score

def negative_sampling(edge_index, num_neg_samples, num_nodes):
    # Generate random node pairs that are not connected
    neg_edges = set()
    existing_edges = set([(edge_index[0, i].item(), edge_index[1, i].item()) for i in range(edge_index.size(1))])
    while len(neg_edges) < num_neg_samples:
        u = random.randint(0, num_nodes - 1)
        v = random.randint(0, num_nodes - 1)
        if u != v and (u, v) not in existing_edges and (v, u) not in existing_edges:
            neg_edges.add((u, v))
    neg_edges = torch.tensor(list(neg_edges), dtype=torch.long).t()
    return neg_edges

def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    z = model(data.x, data.edge_index)
    
    # Positive edges
    pos_edge = data.train_edges.t()
    pos_pred = model.decode(z, pos_edge)
    pos_label = torch.ones(pos_pred.size(0))
    
    # Negative edges: Sample equal number of non-edges
    num_neg = pos_pred.size(0)
    neg_edges = negative_sampling(data.edge_index, num_neg, data.num_nodes)
    neg_pred = model.decode(z, neg_edges)
    neg_label = torch.zeros(neg_pred.size(0))
    
    # Combine
    pred = torch.cat([pos_pred, neg_pred], dim=0)
    labels = torch.cat([pos_label, neg_label], dim=0)
    
    loss = criterion(pred, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

def test(model, data):
    model.eval()
    with torch.no_grad():
        z = model(data.x, data.edge_index)
        
        # Positive test edges
        pos_edge = data.test_edges.t()
        pos_pred = model.decode(z, pos_edge).cpu().numpy()
        pos_label = np.ones(pos_pred.shape[0])
        
        # Negative test edges
        neg_edge = data.test_neg_edges.t()
        neg_pred = model.decode(z, neg_edge).cpu().numpy()
        neg_label = np.zeros(neg_pred.shape[0])
        
        # Combine
        preds = np.concatenate([pos_pred, neg_pred])
        labels = np.concatenate([pos_label, neg_label])
        
        auc = roc_auc_score(labels, preds)
        ap = average_precision_score(labels, preds)
        
    return auc, ap


In [5]:
# Initialize model, optimizer, and loss function
num_features = data.num_features
hidden_dim = 64
model = GNN15Layer(num_features, hidden_dim)
optimizer = Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

# Training loop
epochs = 100
for epoch in range(1, epochs + 1):
    loss = train(model, data, optimizer, criterion)
    if epoch % 10 == 0 or epoch == 1:
        auc, ap = test(model, data)
        print(f"Epoch {epoch}: Loss={loss:.4f}, AUC={auc:.4f}, AP={ap:.4f}")


Epoch 1: Loss=0.6854, AUC=0.6526, AP=0.6462
Epoch 10: Loss=0.6866, AUC=0.6472, AP=0.6405
Epoch 20: Loss=0.6898, AUC=0.6348, AP=0.6326
Epoch 30: Loss=0.6924, AUC=0.6279, AP=0.6371
Epoch 40: Loss=0.7015, AUC=0.6378, AP=0.6452
Epoch 50: Loss=0.6899, AUC=0.6551, AP=0.6485
Epoch 60: Loss=0.6935, AUC=0.6417, AP=0.6409
Epoch 70: Loss=0.6891, AUC=0.6402, AP=0.6333
Epoch 80: Loss=0.6916, AUC=0.6294, AP=0.6349
Epoch 90: Loss=0.6956, AUC=0.6215, AP=0.6302
Epoch 100: Loss=0.6911, AUC=0.6202, AP=0.6290


In [6]:
# Save the trained model
torch.save(model.state_dict(), 'gnn15layer_model.pth')
print("Model saved as 'gnn15layer_model.pth'")


Model saved as 'gnn15layer_model.pth'


In [7]:
# Initialize a new model instance
loaded_model = GNN15Layer(num_features, hidden_dim)
loaded_model.load_state_dict(torch.load('gnn15layer_model.pth'))
loaded_model.eval()
print("Model loaded successfully.")


Model loaded successfully.


In [ ]:
import flask
from flask import Flask, request, jsonify
import torch
from threading import Thread

# Initialize Flask app
app = Flask(__name__)

# Load the trained model
loaded_model = GNN15Layer(num_features, hidden_dim)
loaded_model.load_state_dict(torch.load('gnn15layer_model.pth'))
loaded_model.eval()

# Precompute node embeddings
with torch.no_grad():
    z = loaded_model(data.x, data.edge_index)

@app.route('/predict', methods=['POST'])
def predict():
    """
    Expects JSON with 'node1' and 'node2' as integers.
    Example:
    {
        "node1": 0,
        "node2": 1
    }
    """
    data_json = request.get_json()
    node1 = data_json.get('node1')
    node2 = data_json.get('node2')
    
    num_nodes = data.num_nodes
    if node1 is None or node2 is None:
        return jsonify({"error": "Please provide 'node1' and 'node2' in the JSON payload."}), 400
    if not (0 <= node1 < num_nodes) or not (0 <= node2 < num_nodes):
        return jsonify({"error": f"node indices must be between 0 and {num_nodes -1}"}), 400
    
    edge_pair = torch.tensor([[node1, node2]], dtype=torch.long).t()
    with torch.no_grad():
        score = loaded_model.decode(z, edge_pair).item()
    
    prediction = {
        "node1": node1,
        "node2": node2,
        "edge_probability": score
    }
    
    return jsonify(prediction)

@app.route('/')
def home():
    return '''
    <h1>GNN Edge Prediction</h1>
    <p>Use the /predict endpoint with a JSON payload containing "node1" and "node2".</p>
    <p>Example payload:</p>
    <pre>
    {
        "node1": 0,
        "node2": 1
    }
    </pre>
    '''

def run_flask():
    app.run(host='0.0.0.0', port=5000)

# Run Flask in a separate thread
flask_thread = Thread(target=run_flask)
flask_thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.2.105:5000
Press CTRL+C to quit
192.168.2.105 - - [25/Nov/2024 13:13:22] "GET / HTTP/1.1" 200 -
192.168.2.105 - - [25/Nov/2024 13:13:22] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/Nov/2024 13:13:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2024 13:13:25] "GET /favicon.ico HTTP/1.1" 404 -
